# Reto 4

In [1]:
#Importar librerías
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from random import randrange

Se leen los datos utilizando readcsv de Pandas

In [2]:
#Leer el archivo con los datos
data_wine = pd.read_csv('./data/winequality-white.csv',sep=';')

Se separan los datos en "x" y "y". Se normaliza x

In [3]:
#La variable dependiente es la última columna, las independientes son las anteriores
x= data_wine.iloc[:,0:11]
y= data_wine.iloc[:,11:12]

Se parten los datos en entrenamiento y test

In [4]:
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 2000, random_state = 0)

In [5]:
#Se concatenan los datos de test
newData= pd.concat([xTrain,yTrain], axis = 1)

Se escogen aleatoriamente diferentes tamaños de datos para entrenar

In [6]:
#De estos datos concatenados se escogen aleatoriamente 100,1000 y 2898 para diferentes modelos. 
#De aquí se vuelven a separar en x y y
dataTrain1= newData.sample(100, random_state = 0)
xTrain1= dataTrain1.iloc[:,0:11]
yTrain1= dataTrain1.iloc[:,11]
dataTrain2= newData.sample(1000, random_state = 0)
xTrain2= dataTrain2.iloc[:,0:11]
yTrain2= dataTrain2.iloc[:,11]
dataTrain3= newData.sample(2898, random_state = 0)
xTrain3= dataTrain3.iloc[:,0:11]
yTrain3= dataTrain3.iloc[:,11]

Esta función calcula la suma de los errores cuadráticos para obtener un estimativo del error

In [7]:
#Función que calcula el error cuadrático medio a partir de ciertos datos y el vector de parámetros w
def calc_errors(x,y,w):
    err=0
    for i in range(0,x.shape[0]):
        err+=(y[i]-(x[i]*w).sum())**2
    return err

Se definen la función necesaria para realizar descenso de gradiente estocásico

In [8]:
#Función que calcula el gradiente descendente, demora al correr
#numErrors es el número de errores consecutivos cuya media debe ser menor que el threshold 
#para que se asuma que el algoritmo terminó, este se escogió con ensayo y error.
#El threshold se define por la variable minError, este se escogió con ensayo y error.
def gradient_desc(nX,nY,numCiclos=29,minError=0.0001):
    a=nX.to_numpy()
    y=nY.to_numpy()
    #Se concatena un 1 al final de las variables para representar la constante w0
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    #Se calcula la hessiana
    H=[[0] * wNumber]* wNumber
    for i in range(0, numData):
        xi= x[i]
        H+=xi.transpose()*xi
    ei=np.linalg.eigvals(H)
    #Se obtienen los valores propios y se calcula n como 20/lambdaMax, valor escogido con ensayo y error. 
    #Si se utiliza 2/lambdaMax que es lo que indica la teoría el algoritmo demora mucho en converger
    n=20/max(ei)
    ws= [0.01] * wNumber
    errorA=1
    errorP=0
    j=0
    #Variable que representa la media mínima del error cuadrático medio actual
    minT=10
    #El ciclo se ejecuta hasta que el error cuadrático medio sea similar al error cuadrático medio
    #de los numCiclos ciclos anteriores a este. Así se asume que el modelo convergió
    while minT > minError:
        #Se escoge una fila de datos aleatoriamente
        i= randrange(numData)
        #Se calcula ws con el algoritmo de descenso de gradiente estocástico
        g=(x[i]*ws).sum()
        e=(g-y[i])         
        ws= ws - n*e*x[i]
        j=(j+1)%numCiclos
        #Cada numCiclos ciclos se pregunta si el error se similar al error de hace numCiclos ciclos
        if j==0:
            errorP=errorA
            errorA=calc_errors(x,y,ws)
            #Se guarda solo el error mínimo, que es el que se usa para comparar
            if abs(errorP-errorA)< minT:
                minT=abs(errorP-errorA)
                #Se puede descomentar el siguiente print para observar como este valor disminuye
                #print(minT)
    return ws
    

Se entrenan los modelos

In [9]:
#Cálculo del modelo 1
linearRegr1 = gradient_desc(xTrain1, yTrain1)
print(linearRegr1)

[ 1.89975079e-01  1.22958122e-02  1.81062079e-02 -7.81775202e-03
  1.05618744e-02  7.21985975e-03 -1.74395681e-04  3.95074848e-02
  1.07534485e-01  2.33471628e-02  3.74380496e-01  3.98625725e-02]


In [10]:
#Cálculo del modelo 2
linearRegr2 = gradient_desc(xTrain2, yTrain2)
print(linearRegr2)

[ 0.10442256 -0.00568421  0.01633081  0.01998139  0.0099582   0.01076732
 -0.000943    0.04427826  0.13924648  0.02861502  0.40389814  0.04474297]


In [11]:
#Cálculo del modelo 3
linearRegr3 = gradient_desc(xTrain3, yTrain3)
print(linearRegr3)

[ 0.16446946  0.01294825  0.01758163  0.0172221   0.01053063  0.0123741
 -0.00173209  0.0386495   0.10669988  0.02467593  0.38089398  0.03900927]


Se construye una función de éxito para evaluar el modelo.

In [12]:
#Función para calcular R^2 como 1-u/v, similar a como lo calcula sklearn para validar
def calc_r2(nX,nY,w):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    u=0
    v=0
    #ym es la media de todos los y
    ym= y.mean()
    err=0
    for i in range(0, numData):
        xi=x[i]
        yi=y[i]
        yp=(xi*w).sum()
        #u es la suma de cuadrados residuales (yi - yPredecido)
        u+=(yi-yp)**2
        #u es la suma total de cuadrados (yi - yMedia)
        v+=(yi-ym)**2
    r2=1-(u/v)
    return r2
    

Se evalúa el modelo con la función personalizada de éxito

In [13]:
#Cálculo de R^2 para el modelo 1
r21 = calc_r2(xTest, yTest,linearRegr1)
print(r21)

[0.12465322]


In [14]:
#Cálculo de R^2 para el modelo 2
r22 = calc_r2(xTest, yTest,linearRegr2)
print(r22)

[0.16348888]


In [15]:
#Cálculo de R^2 para el modelo 3
r23 = calc_r2(xTest, yTest,linearRegr3)
print(r23)

[0.14452474]


# Resultados
Se puede observar que el R^2 de todos los modelos es bastante bajo. Este valor es incluso más bajo que el valor de los modelos de sklearn. Al igual que en el reto 3 esto puede explicarse con el hecho que no se calculó el mínimo exacto, sino una aproximación utilizando descenso de gradiente. El modelo con mayor R^2 es el segundo, lo cual justifica en parte que los datos se comportan de una manera no lineal ya que el modelo con menos datos tuvo un mejor ajuste que el que tiene más datos (además de ser dar como resultado todos los modelos un valor bastante bajo de R^2).